In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
# df.corr()
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

Goal of the project is hyperperameter tunin, not to build model


In [5]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()  # create an object of the class

X = scaler.fit_transform(X)
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)

In [44]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [9]:
model = Sequential()
model.add(Dense(32, activation="relu", input_dim=8))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="Adam", loss="binary_crossentropy",
              metrics=["accuracy"])

In [10]:
model.fit(X_train, y_train, batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 13ms/step - loss: 0.7942 - accuracy: 0.4511 - val_loss: 0.7334 - val_accuracy: 0.4805
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7134 - accuracy: 0.5375 - val_loss: 0.6728 - val_accuracy: 0.5714
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6582 - accuracy: 0.5863 - val_loss: 0.6275 - val_accuracy: 0.6104
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6200 - accuracy: 0.6336 - val_loss: 0.5930 - val_accuracy: 0.7013
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5903 - accuracy: 0.6775 - val_loss: 0.5680 - val_accuracy: 0.7468
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5681 - accuracy: 0.6857 - val_loss: 0.5474 - val_accuracy: 0.7208
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5491 - accuracy: 0.7182 - val_loss: 0.5323 - val_accuracy: 0.7403
Epoch 8/100


# Use of keras-tuner for the automations


### How to select appropiate optimizer


In [11]:
import kerastuner as kt

Using TensorFlow backend


C:\Users\dbhuva26\AppData\Local\Temp\ipykernel_21308\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [12]:
# First we make a function and then tuner, after we pass that function into tuner object

def build_model(hp):
    model = Sequential()

    model.add(Dense(32, activation="relu", input_dim=8))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = hp.Choice("optimizer", values=[
                          "adam", "sgd", "rmsprop", "adadelta"])

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [13]:
# Kind of greedSearchCV
tuner = kt.RandomSearch(build_model, objective="val_accuracy",
                        max_trials=5, directory="Tuner dir", project_name="Optimizer")

In [14]:
# Here, we keep less epochs since here we are selecting only model here not model building. so after getting best model we can train it accordingly
tuner.search(X_train, y_train, epochs=5, validation_data=[X_test, y_test])

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 07s


In [15]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [16]:
# Get that model, num_models = <number of model we want>
best_model = tuner.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(
    X_test, y_test))  # Model has already run 5 epochs, so starting from 6

Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4177 - accuracy: 0.8013 - val_loss: 0.4484 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4176 - accuracy: 0.7948 - val_loss: 0.4479 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4169 - accuracy: 0.8062 - val_loss: 0.4482 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4168 - accuracy: 0.8094 - val_loss: 0.4465 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4165 - accuracy: 0.8046 - val_loss: 0.4459 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4164 - accuracy: 0.8094 - val_loss: 0.4479 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4160 - accuracy: 0.8127 - val_loss: 0.4502 - val_accuracy: 0.8117
Epoch 14/

### finding Number of nurons using kearas-tuner


In [19]:
def build_model(hp):
    model = Sequential()

    units = hp.Int("units", min_value=8, max_value=128, step=4)

    model.add(Dense(units=units, activation="relu", input_dim=8))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [20]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy",
                        max_trials=5, directory="Tuner dir", project_name="Neurons")

In [21]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 08s


In [22]:
tuner.get_best_hyperparameters()[0].values

{'units': 116}

In [23]:
model = tuner.get_best_models(num_models=1)[0]

In [24]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(
    X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5223 - accuracy: 0.7655 - val_loss: 0.4991 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4979 - accuracy: 0.7752 - val_loss: 0.4807 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4810 - accuracy: 0.7850 - val_loss: 0.4727 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4721 - accuracy: 0.7834 - val_loss: 0.4668 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4659 - accuracy: 0.7850 - val_loss: 0.4642 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4595 - accuracy: 0.7866 - val_loss: 0.4622 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4574 - accuracy: 0.7866 - val_loss: 0.4615 - val_accuracy: 0.7727
Epoch 14

### Selecting number of layers using Keras-tuner


In [25]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(72, activation="relu", input_dim=8))

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        model.add(Dense(72, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [26]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy",
                        max_trials=5, directory="Tuner dir", project_name="layers")

In [27]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 11s


In [28]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [29]:
model = tuner.get_best_models(num_models=1)[0]

In [30]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(
    X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4479 - accuracy: 0.7866 - val_loss: 0.5067 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4433 - accuracy: 0.7834 - val_loss: 0.4913 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4302 - accuracy: 0.7980 - val_loss: 0.5269 - val_accuracy: 0.8312
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4254 - accuracy: 0.8046 - val_loss: 0.4934 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4236 - accuracy: 0.8094 - val_loss: 0.5421 - val_accuracy: 0.7403
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4045 - accuracy: 0.8225 - val_loss: 0.4995 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3882 - accuracy: 0.8322 - val_loss: 0.5168 - val_accuracy: 0.8182
Epoch 14

In [37]:
# All dynamic paramter in one

def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                      activation=hp.Choice("activation" + str(i), values=["relu", "tanh"]), input_dim=8))
        else:
            model.add(Dense(hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                      activation=hp.Choice("activation" + str(i), values=["relu", "tanh"])))

        counter += 1

    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer=hp.Choice("optimizer", values=[
                  "rmsprop", "adam", "sgd", "nadam"]), loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [38]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy",
                        max_trials=5, directory="Tuner dir", project_name="all customs")

In [39]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 13s


In [40]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 32,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 96,
 'activation1': 'relu',
 'units2': 104,
 'activation2': 'relu',
 'units3': 16,
 'activation3': 'relu',
 'units4': 72,
 'activation4': 'tanh',
 'units5': 120,
 'activation5': 'tanh',
 'units6': 112,
 'activation6': 'tanh',
 'units7': 16,
 'activation7': 'tanh'}

In [42]:
# Creating a model using above hyper paramters

model = tuner.get_best_models(num_models=1)[0]

model.fit(X_train, y_train, epochs=50, initial_epoch=5, validation_data=(
    X_test, y_test))

Epoch 6/50
20/20 [==============================] - 1s 14ms/step - loss: 0.4549 - accuracy: 0.7687 - val_loss: 0.4659 - val_accuracy: 0.7857
Epoch 7/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4506 - accuracy: 0.7704 - val_loss: 0.4615 - val_accuracy: 0.8052
Epoch 8/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4488 - accuracy: 0.7655 - val_loss: 0.4656 - val_accuracy: 0.7987
Epoch 9/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4462 - accuracy: 0.7769 - val_loss: 0.4687 - val_accuracy: 0.7857
Epoch 10/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4442 - accuracy: 0.7671 - val_loss: 0.4715 - val_accuracy: 0.7922
Epoch 11/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4425 - accuracy: 0.7769 - val_loss: 0.4745 - val_accuracy: 0.7857
Epoch 12/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4410 - accuracy: 0.7785 - val_loss: 0.4745 - val_accuracy: 0.7922
Epoch 13/50
20/2

In [45]:
# Adding dropout into model


def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                      activation=hp.Choice("activation" + str(i), values=["relu", "tanh"]), input_dim=8))
            model.add(Dropout(hp.Choice("dropout" + str(i),
                      values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
        else:
            model.add(Dense(hp.Int("units" + str(i), min_value=8, max_value=128, step=8),
                      activation=hp.Choice("activation" + str(i), values=["relu", "tanh"])))
            model.add(Dropout(hp.Choice("dropout" + str(i),
                      values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))

        counter += 1

    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer=hp.Choice("optimizer", values=[
                  "rmsprop", "adam", "sgd", "nadam"]), loss="binary_crossentropy", metrics=["accuracy"])

    return model

In [46]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy",
                        max_trials=5, directory="Tuner dir", project_name="all customs with dropout")

In [47]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 16s


In [48]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 72,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'nadam',
 'units1': 112,
 'activation1': 'tanh',
 'dropout1': 0.6,
 'units2': 16,
 'activation2': 'tanh',
 'dropout2': 0.7,
 'units3': 96,
 'activation3': 'relu',
 'dropout3': 0.9,
 'units4': 112,
 'activation4': 'relu',
 'dropout4': 0.7,
 'units5': 112,
 'activation5': 'relu',
 'dropout5': 0.8,
 'units6': 128,
 'activation6': 'relu',
 'dropout6': 0.7,
 'units7': 16,
 'activation7': 'relu',
 'dropout7': 0.8,
 'units8': 56,
 'activation8': 'tanh',
 'dropout8': 0.7}

In [49]:
model = tuner.get_best_models(num_models=1)[0]

model.fit(X_train, y_train, epochs=50, initial_epoch=5, validation_data=(
    X_test, y_test))

Epoch 6/50
20/20 [==============================] - 2s 12ms/step - loss: 0.4976 - accuracy: 0.7492 - val_loss: 0.4686 - val_accuracy: 0.7857
Epoch 7/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4937 - accuracy: 0.7573 - val_loss: 0.4654 - val_accuracy: 0.7792
Epoch 8/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4836 - accuracy: 0.7492 - val_loss: 0.4676 - val_accuracy: 0.7792
Epoch 9/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4763 - accuracy: 0.7606 - val_loss: 0.4684 - val_accuracy: 0.7857
Epoch 10/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4979 - accuracy: 0.7687 - val_loss: 0.4645 - val_accuracy: 0.7857
Epoch 11/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4847 - accuracy: 0.7655 - val_loss: 0.4661 - val_accuracy: 0.7922
Epoch 12/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4927 - accuracy: 0.7638 - val_loss: 0.4681 - val_accuracy: 0.7857
Epoch 13/50
20/2